## USE Model

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas
from sklearn.metrics import confusion_matrix,precision_score,recall_score,roc_curve,auc,f1_score,accuracy_score
import numpy as np


candidate_labels = ["finance", "apparel and fashion", "food and dining", "health and medicine", 
                    "tech and electronics", "entertainment", "home and garden", "science and environment",
                    "government and politics", "apparel", ]
                    # "career", "banking and payments", 
                    # "sports", "personal care and beauty", "travel", "real estate", 
                    # "investment", "automotive", "consumer electronics", "social", 
                    # "sports fan", "restaurants and dining", "shoes", "casual apparel", 
                    # "cooking and recipes", "education" ]

2023-06-12 18:18:33.215589: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarni

In [2]:
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

2023-06-12 18:18:42.428182: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-12 18:18:42.434348: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-12 18:18:42.435969: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
input_embedding = model(["Who are you voting in 2020?"])[0]

label_embeddings = model(candidate_labels)
scores = tf.reduce_sum(input_embedding * label_embeddings, axis=1)

# label_scores = dict(zip(candidate_labels, scores.numpy()))
scores.numpy()


array([ 0.029844  , -0.05816384, -0.07609773, -0.02770378, -0.05365713,
       -0.06250247, -0.12026697, -0.0209319 ,  0.08021943, -0.06764548],
      dtype=float32)

In [31]:
def get_df(filename, headers):
    df_read = pandas.read_csv(filename, delimiter='\x01', header=0, names=headers, error_bad_lines=False)
    df = df_read[df_read['snippet'].apply(lambda x: isinstance(x,str) )]
    return df


def get_df_with_prediction(count, df, inp_columns):
    small_df = df.take([x for x in range(0,count)])
    arr_pred = []
    it = 1
    for id in small_df.index:
        print(it, end = " ")
        it +=1
        sequence = " ".join([small_df[col][id] for col in inp_columns])
        input_embedding = model([sequence])[0]
        label_embeddings = model(candidate_labels)
        scores = tf.reduce_sum(input_embedding * label_embeddings, axis=1)
        arr_pred.append(scores.numpy())
    small_df['pred'] = arr_pred
    return small_df


def metrics(topic, y_true, y_pred, y_prob=None):        
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    metric_map = {
        'topic': topic,
        'precision': precision, 
        'recall': recall, 
        'f1_score': f1, 
        'accuracy': acc,        
        'true negative':tn, 
        'false positive': fp,
        'false negative': fn,
        'true positive': tp,

    }
    if y_prob is not None:
        fpr, tpr, thresholds = roc_curve(y_true, y_prob)
        auc_score = auc(fpr, tpr)
        metric_map['auc'] = auc_score
    return metric_map
    

def evaluate(small_df, label_col="label", threshold=0.0):
    result = {}
    for topic in candidate_labels:
        result[topic] = {"y_true":[], "y_pred":[], "y_pred_1":[], "y_prob":[]}
    for id in small_df.index:
        labels = [x.replace("_"," ") for x in small_df[label_col][id].split("\x02")]
        pred = small_df["pred"][id]
        for i,topic in enumerate(candidate_labels):
            result[topic]["y_true"].append(1 if topic in labels else 0)
            result[topic]["y_prob"].append(pred[i])
            result[topic]["y_pred"].append(pred[i]>threshold)                
                
                
    eval_result = []
    eval_result_1 = []    
    for topic in result.keys():
        eval_result.append(
            metrics(
                topic,
                result[topic]["y_true"], 
                result[topic]["y_pred"], 
                result[topic]["y_prob"]
            )
        )
    eval_df = pandas.DataFrame(eval_result)
    return eval_df

In [6]:
df_2 = get_df("sample2", ['xcluster','guid','immutable_id','subject','fromname','content','snippet','topics','rnk'])

/var/tmp/ipykernel_212723/2937360642.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_read = pandas.read_csv(filename, delimiter='\x01', header=0, names=headers, error_bad_lines=False)
Skipping line 1786: expected 9 fields, saw 10
Skipping line 9495: expected 9 fields, saw 10
Skipping line 25163: expected 9 fields, saw 11
Skipping line 31456: expected 9 fields, saw 12



In [ ]:
small_df_2 = get_df_with_prediction(10000, df_2, ["subject","fromname","content"])

In [33]:
eval_df = evaluate(small_df_2[small_df_2['topics'].apply(lambda x: isinstance(x,str))], label_col="topics", threshold=0.01)

eval_df\
.style\
.format(precision=2) \
.set_properties(**{'color': 'red'}, subset=['false negative', 'false positive'])\
.set_properties(**{'color': 'green'}, subset=['true negative', 'true positive'])

,topic,precision,recall,f1_score,accuracy,true negative,false positive,false negative,true positive,auc
0,finance,0.51,0.78,0.62,0.84,7170,1204,352,1272,0.90
1,apparel and fashion,0.21,0.91,0.34,0.53,4080,4614,113,1191,0.85
2,food and dining,0.13,0.92,0.23,0.50,4302,4901,66,729,0.85
3,health and medicine,0.12,0.89,0.20,0.46,3958,5265,89,686,0.80
4,tech and electronics,0.11,0.73,0.19,0.55,4955,4306,200,537,0.72
5,entertainment,0.21,0.30,0.25,0.87,8491,787,506,214,0.64
6,home and garden,0.09,0.95,0.16,0.32,2554,6746,33,665,0.80
7,science and environment,0.02,0.13,0.04,0.61,6026,3353,540,79,0.32
8,government and politics,0.12,0.75,0.20,0.64,5925,3459,154,460,0.76
9,apparel,0.12,0.95,0.22,0.61,5567,3858,29,544,0.92


In [23]:
small_df_2.to_pickle("small_df_use_10000")

In [ ]:
small_df_2[small_df_2['pred'].apply(lambda x: x[3]>0.1)]